In [1]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 708.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninsta

In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 6.9 MB/s eta 0:00:00


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="TcSwu1iRXemlvskZKYM1")
project = rf.workspace().project("numbers-identification")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

In [6]:
from torchmetrics.text import CharErrorRate

In [7]:
cer = CharErrorRate()
def calculate_metric(preds, target):
  return cer(preds, target)

In [8]:
files = []
labels = []

In [9]:
for filename in os.listdir('/content/'):
  if filename.endswith('jpg'):
    labels.append(filename.split('_')[0])
    files.append('/content/' + filename)

In [10]:
files[0:5]

['/content/KR75XO3_1__png_jpg.rf.1c359888dd480d6b8fa2cef1d23bc431.jpg',
 '/content/9JK9DXP_1__png_jpg.rf.99361ad92a7c0e4117ee80a55ab10b8d.jpg',
 '/content/52L6C2F_1__png_jpg.rf.ebf27e1836d4687bdff64018bcc92ec3.jpg',
 '/content/VRTK6F6_1__png_jpg.rf.bc05da420af6b8bc59470a889c642481.jpg',
 '/content/JNJGSAI_1__png_jpg.rf.586ddc08be8ea4989c5dd055f58da3f5.jpg']

In [11]:
labels[0:5]

['KR75XO3', '9JK9DXP', '52L6C2F', 'VRTK6F6', 'JNJGSAI']

In [13]:
prediction = model.predict("/content/02R2JNV_1__png_jpg.rf.962717fcecf3b8e5697ab99ab4642320.jpg", confidence=40, overlap=30).json()

In [15]:
predictions = sorted(prediction['predictions'], key=lambda d: d['x'])

In [16]:
pred = [x['class'].replace('-', '') for x in predictions]

In [17]:
"".join(pred)

'02R2JNV'

In [18]:
total_length = 0
total_score = 0
mean_cer = 0

In [19]:
for file, label in zip(files, labels):
  prediction = model.predict(file, confidence=40, overlap=30).json()
  prediction = sorted(prediction['predictions'], key=lambda d: d['x'])
  pred = [x['class'].replace('-', '') for x in prediction]
  predicted_label = "".join(pred)
  length = len(label)
  total_length += length
  score = calculate_metric(predicted_label, label)
  total_score += score * length
  mean_cer += score

In [20]:
print(mean_cer/len(files))

tensor(0.0158)


In [21]:
len(files)

263

In [22]:
print(total_score/total_length)

tensor(0.0158)


In [23]:
total_length

1841